In [1]:
import pandas as pd
import networkx as nx  # pip install --user networkx

## Load data

In [2]:
nodes = pd.read_pickle("cachenodes.pkl")
edges = pd.read_pickle("edges.pkl")
comp_nodes = pd.read_pickle("comp_nodes.pkl")

## Build topology

In [3]:
def build_topology(nodes, edges):
    topology = nx.Graph()
    
    # add all nodes
    for index, row in nodes.iterrows():
        node_name = row["name"]
        node_attributes = row.drop(["name"]).to_dict()
        topology.add_node(node_name, attr_dict=node_attributes)
    
    # add all edges
    for index, row in edges.iterrows():
        node1_name = row["node1"]
        node2_name = row["node2"]
        edge_attributes = row.drop(["node1", "node2"]).to_dict()
        topology.add_edge(node1_name, node2_name, attr_dict=edge_attributes)
    
    return topology

In [4]:
topology = build_topology(nodes, edges)

## Calculate shortest path for every pair of computational nodes

### Helper function for caching of results

It allows the program to save calculated tables or other objects
or load them from disk if they are already there

In [9]:
from libcrap.core import calcsave_or_load
from functools import partial

In [10]:
pd_diskcache = partial(calcsave_or_load, load_func=pd.read_pickle, save_func=pd.to_pickle)

### Actually do the work

In [12]:
import itertools

In [13]:
@pd_diskcache("paths.pkl")
def find_comp_to_comp_shortest_paths(topology, comp_nodes):
    paths_ugly = nx.all_pairs_shortest_path(topology)
    # calculates shortest paths and stores them in a dict of dicts
    
    # build a table with all computational node pairs
    # they are not duplicated
    # if there is ("n48001", "n49419") then there is no ("n49419", "n48001") pair
    comp_node_pairs = pd.DataFrame.from_records(
        itertools.chain.from_iterable(
            [(node1, node2) for node2 in comp_nodes.iloc[index+1:]]
            for (index, node1) in comp_nodes.iteritems()
        ),
        columns=["node1", "node2"]
    )

    # write shortest paths to this table
    comp_node_pairs["shortest_path"] = comp_node_pairs.apply(
        lambda row: paths_ugly[row.loc["node1"]][row.loc["node2"]],
        axis=1
    )
    return comp_node_pairs

In [14]:
# shortest paths between all computational nodes
paths = find_comp_to_comp_shortest_paths(topology, comp_nodes)

## Calculate feature lists of these paths

### Helper functions

In [18]:
topology.node["КГК.48.0.3"].items()

dict_items([('type_', 'switch')])

In [20]:
footopology = nx.Graph()
footopology.add_node("kek", attr_dict={"a": 1, "b": "lol"})
tuple(footopology.node["kek"].items())

(('a', 1), ('b', 'lol'))

In [21]:
def get_node_properties(topology, node):
    """Returns node properties as a tuple of tuples.
    
    >>> some_topology = nx.Graph()
    >>> some_topology.add_node("kek", attr_dict={"a": 1, "b": "lol"})
    >>> get_node_properties(some_topology, "kek")
    (('a', 1), ('b', 'lol'))
    """
    return tuple(topology.node[node].items())

### Test helper functions

In [24]:
import doctest

In [27]:
def test_get_node_properties():
    doctest.run_docstring_examples(get_node_properties, globals())
    assert get_node_properties(topology, "КГК.48.0.3") == (("type_", "switch"),)

In [28]:
test_get_node_properties()